In [1]:
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
from bs4 import BeautifulSoup
import time
import pandas as pd
from dotenv import load_dotenv
import os
import sqlalchemy

In [3]:
def obter_dados_pagina(driver, url):
    """Extrai os dados da tabela de uma página específica."""
    driver.get(url)
    time.sleep(5)  

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    wrapper_div = soup.find('div', {'id': 'DataTables_Table_0_wrapper'})
    if not wrapper_div:
        print("Wrapper da tabela não encontrado!")
        return [], None

    table = wrapper_div.find('table', {'id': 'DataTables_Table_0'})
    if not table:
        print("Tabela não encontrada!")
        return [], None

    headers = []
    for th in table.find('thead').find_all('th'):
        span = th.find('span')
        if span:
            headers.append(span.text.strip())

    dados = []
    for row in table.find('tbody').find_all('tr'):
        row_data = []
        # Extrair dados de cada td
        for td in row.find_all('td'):
            cell_value = td.text.strip()
            row_data.append(cell_value)
            
        dados.append(row_data)

    return dados, headers

def navegar_proxima_pagina(driver):
    """Navega para a próxima página, retornando se o clique foi bem-sucedido ou não."""
    try:
        next_button = driver.find_element("xpath", "//div[@id='DataTables_Table_0_wrapper']//button[contains(@class, 'dt-paging-button next')]")
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        time.sleep(1)  

        next_button.click()  # Tenta o clique
        time.sleep(3) 
        return True  
    except ElementClickInterceptedException:
        print("Elemento 'Próxima' está bloqueado por outro elemento.")
        time.sleep(1)  
        return False
    except NoSuchElementException:
        print("Botão 'Próxima' não encontrado. Parando o loop.")
        return False  

def extrair_dados(url, limite_paginas):
    """Função principal que coleta dados de várias páginas."""
    dados_todas_paginas = []  
    pagina_atual = 1

    # Inicializar o driver do Selenium
    driver = webdriver.Chrome()

    while pagina_atual <= limite_paginas:
        dados, headers = obter_dados_pagina(driver, url)

        if dados and headers:
            dados_todas_paginas.extend(dados)  

        pagina_atual += 1  

    driver.quit()  # Encerra o driver do Selenium

    return dados_todas_paginas, headers

In [5]:
# Definir os parâmetros
url = "https://steamdb.info/sales/"
limite_paginas = 1

dados_todas_paginas, headers = extrair_dados(url, limite_paginas)

# Criar DataFrame a partir dos dados coletados
df = pd.DataFrame(dados_todas_paginas, columns=headers)

df.columns = ['Index', 'Unused', 'Name', 'Discount %', 'Price', 'Rating', 'Release','Ends', 'Started']

df = df.drop(columns=['Index', 'Unused', 'Ends', 'Started'])

# Exibindo os dados formatados
df

,Name,Discount %,Price,Rating,Release
0,Pistol Whip\n\nPlay For Free\nall-time low: R$...,-50%,"R$ 44,49",90.39%,Nov 2019
1,Age of Wonders 4\n\nPlay For Free\nall-time lo...,-35%,"R$ 129,99",79.48%,May 2023
2,"ICARUS\n\nPlay For Free\nall-time low: R$ 44,2...",-50%,"R$ 50,99",70.30%,Dec 2021
3,Age of Water\n\nPlay For Freenew historical low,-50%,"R$ 74,50",62.27%,Apr 2024
4,Baldur's Gate 3\n\nWeek Long DealTop Seller,-20%,"R$ 159,99",95.98%,Aug 2023
...,...,...,...,...,...
95,A Perfect Day\n\nnew historical low,-41%,"R$ 35,39",81.25%,Feb 2022
96,The Lord of the Rings: Return to Moria™\n\nTop...,-33%,"R$ 49,57",79.05%,Aug 2024
97,Teenage Mutant Ninja Turtles: Splintered Fate\...,-10%,"R$ 58,49",78.96%,Nov 2024
98,Sengoku Dynasty\n\nIntroductory OfferTop Selle...,-20%,"R$ 71,19",71.90%,Nov 2024
